In [ ]:
!git clone https://github.com/pp00704831/Stripformer.git
%cd Stripformer
!conda create -n Stripformer python=3.6 -y
!source activate Stripformer
!conda install pytorch==1.8.0 torchvision==0.9.0 torchaudio==0.8.0 cudatoolkit=11.1 -c pytorch -c conda-forge -y
!pip install opencv-python tqdm pyyaml joblib glog scikit-image tensorboardX albumentations
!pip install -U albumentations[imgaug]
!pip install albumentations==1.1.0

Cloning into 'Stripformer'...
remote: Enumerating objects: 155, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 155 (delta 63), reused 25 (delta 25), pack-reused 88
Receiving objects: 100% (155/155), 150.71 KiB | 18.84 MiB/s, done.
Resolving deltas: 100% (78/78), done.
/content/Stripformer
/bin/bash: line 1: conda: command not found
/bin/bash: line 1: activate: No such file or directory
/bin/bash: line 1: conda: command not found
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-gflags: filename=python_gflags-3.1.2-py3-none-any.whl size=57370 sha256=a3f16e70d45f8f114eae6aad90c208b30e07bac55fda5c988f5474ad33b7a3cd
  Stored in directory: /root/.cache/pip/wheels/0d/7d/a2/e858f0632653a3c752a807f85a39ca9d7ed75796c62845e66c
Successfully built python

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Copy the dataset from my drive to ./datasets/GoPro/test

In [ ]:
!mkdir -p ./datasets/GoPro/test
!cp -r /content/drive/MyDrive/DistortedImages ./datasets/GoPro/test

Download the pretrained model from
https://drive.google.com/drive/folders/18kx_JTj8SzsQk0Wxnkn5_gg_d_iK_LKZ

In [ ]:
!mkdir -p /content/Stripformer
!cp /content/drive/MyDrive/Models/Stripformer_gopro.pth /content/Stripformer/
!cp /content/drive/MyDrive/Models/Stripformer_realblur_J.pth /content/Stripformer/
!cp /content/drive/MyDrive/Models/Stripformer_realblur_R.pth /content/Stripformer/

In [ ]:
%%writefile predict_GoPro_test_results.py
from __future__ import print_function
import numpy as np
import torch
import cv2
import yaml
import os
from torch.autograd import Variable
from models.networks import get_generator
import torchvision
import time
import argparse

def get_args():
    parser = argparse.ArgumentParser('Test an image')
    parser.add_argument('--weights_path', required=True, help='Weights path')
    return parser.parse_args()

if __name__ == '__main__':
    args = get_args()
    with open('config/config_Stripformer_gopro.yaml') as cfg:
        config = yaml.safe_load(cfg)
    blur_path = './datasets/GoPro/test/'
    out_path = '/content/drive/MyDrive/Stripformer_CID'
    if not os.path.isdir(out_path):
        os.mkdir(out_path)
    model = get_generator(config['model'])
    model.load_state_dict(torch.load(args.weights_path))
    model = model.cuda()

    test_time = 0
    iteration = 0
    total_image_number = 256 # Change accordingly

    # warm-up
    warm_up = 0
    print('Hardware warm-up')
    for file in os.listdir(blur_path):
        for img_name in os.listdir(blur_path + '/' + file):
            warm_up += 1
            img = cv2.imread(blur_path + '/' + file + '/' + img_name)
            img_tensor = torch.from_numpy(np.transpose(img / 255, (2, 0, 1)).astype('float32')) - 0.5
            with torch.no_grad():
                img_tensor = Variable(img_tensor.unsqueeze(0)).cuda()
                result_image = model(img_tensor)
            if warm_up == 20:
                break
        break

    for file in os.listdir(blur_path):
        if not os.path.isdir(out_path + '/' + file):
            os.mkdir(out_path + '/' + file)
        for img_name in os.listdir(blur_path + '/' + file):
            img = cv2.imread(blur_path + '/' + file + '/' + img_name)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img_tensor = torch.from_numpy(np.transpose(img / 255, (2, 0, 1)).astype('float32')) - 0.5
            with torch.no_grad():
                iteration += 1
                img_tensor = Variable(img_tensor.unsqueeze(0)).cuda()

                start = time.time()
                result_image = model(img_tensor)
                stop = time.time()
                print('Image:{}/{}, CNN Runtime:{:.4f}'.format(iteration, total_image_number, (stop - start)))
                test_time += stop - start
                print('Average Runtime:{:.4f}'.format(test_time / float(iteration)))
                result_image = result_image + 0.5
                out_file_name = out_path + '/' + file + '/' + img_name
                torchvision.utils.save_image(result_image, out_file_name)

Overwriting predict_GoPro_test_results.py


In [ ]:
!python predict_GoPro_test_results.py --weights_path ./Stripformer_gopro.pth

Hardware warm-up
Image:1/1111, CNN Runtime:0.0403
Average Runtime:0.0403
Image:2/1111, CNN Runtime:0.0476
Average Runtime:0.0439
Image:3/1111, CNN Runtime:0.0253
Average Runtime:0.0377
Image:4/1111, CNN Runtime:0.0251
Average Runtime:0.0346
Image:5/1111, CNN Runtime:0.0248
Average Runtime:0.0326
Image:6/1111, CNN Runtime:0.0268
Average Runtime:0.0316
Image:7/1111, CNN Runtime:0.0452
Average Runtime:0.0336
Image:8/1111, CNN Runtime:0.0348
Average Runtime:0.0337
Image:9/1111, CNN Runtime:0.0261
Average Runtime:0.0329
Image:10/1111, CNN Runtime:0.0248
Average Runtime:0.0321
Image:11/1111, CNN Runtime:0.0252
Average Runtime:0.0315
Image:12/1111, CNN Runtime:0.0259
Average Runtime:0.0310
Image:13/1111, CNN Runtime:0.0303
Average Runtime:0.0309
Image:14/1111, CNN Runtime:0.0403
Average Runtime:0.0316
Image:15/1111, CNN Runtime:0.0241
Average Runtime:0.0311
Image:16/1111, CNN Runtime:0.0345
Average Runtime:0.0313
Image:17/1111, CNN Runtime:0.0248
Average Runtime:0.0309
Image:18/1111, CNN Runt